In [65]:
import json
import requests
import urllib.parse
import pandas as pd 
import io
import re

import warnings                                                                 # Ignore warning related to pandas_profiling
warnings.filterwarnings('ignore') 

pd.set_option('display.max_columns', 100)

In [66]:
def get_transactions(identifier) :
  '''
    Returns the dataframe with the value of we sent as an indetifier
            Parameters:
                    identifier (string): key
                  

            Returns:
                     ts(dataframe): data with given indentifer
   '''

  # Building blocks for the URL

  entrypoint = 'https://sdw-wsrest.ecb.europa.eu/service/' # Using protocol 'https'
  resource = 'data'           # The resource for data queries is always'data'
  flowRef ='BSI'              # Dataflow describing the data that needs to be returned, exchange rates in this case
  key = identifier
  # key = 'Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N'    # Defining the dimension values, explained below

  # Define the parameters
  parameters = {'detail':"dataonly"}
  # Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A
  request_url = entrypoint + resource + '/'+ flowRef + '/' + key

  # Make the HTTP request
  response = requests.get(request_url, params=parameters)

  response = requests.get(request_url, params=parameters, headers={'Accept': 'text/csv'})
  df = pd.read_csv(io.StringIO(response.text))
  
  ts = df.filter(['KEY','TIME_PERIOD', 'OBS_VALUE',], axis=1)
  ts['KEY'] = ts['KEY'].str[4:]
  ts.rename(columns = {'KEY':'IDENTIFIER'}, inplace = True)
  return ts

  
data_frame =  get_transactions('Q.HR.N.A.A20.A.1.AT.2000.Z01.E')


In [67]:
def get_symmetric_identifier(indetifier, swap_comp):
  '''
    Returns the string with the symmetric value
            Parameters:
                    identifier (string): key
                    swap_comp (dictionary) :  swap the values to form symmetric id

            Returns:
                     ts(string): symeetric_id
   '''
  indetifier_list = indetifier.split('.')
  for key, value in swap_comp.items():
      indetifier_list[key],indetifier_list[value] = indetifier_list[value],indetifier_list[key]
  
  asymetric_id = ''.join( i + '.'for i in indetifier_list)[:-1]
  return asymetric_id


identifier = get_symmetric_identifier("Q.HR.N.A.A20.A.1.AT.2000.Z01.E", {1: 7, 2: 3})

In [68]:
def get_asymmetries(indetifier, swap_comp):
  '''
    Returns the string with the symmetric value
            Parameters:
                    identifier (string): key
                    swap_comp (dictionary) :  swap the values to form symmetric id

            Returns:
                     megre_df(dataframe): data with delta of provided_id and symmetric id
   '''
  asymetric_id = get_symmetric_identifier(indetifier, swap_comp)
  provided_id_df = get_transactions(indetifier)
  asymetric_id_df = get_transactions(asymetric_id)
  megre_df = pd.merge(provided_id_df,asymetric_id_df, on = "TIME_PERIOD", how = 'inner')
  megre_df['DELTA'] = megre_df['OBS_VALUE_y'] - megre_df['OBS_VALUE_x']
  megre_df.rename(columns = {'IDENTIFIER_x':'PROVIDED_ID','IDENTIFIER_y': 'SYMMETRIC_ID'}, inplace = True)
  return megre_df.filter(['TIME_PERIOD','PROVIDED_ID','SYMMETRIC_ID','DELTA'])

final_df = get_asymmetries("Q.HR.N.A.A20.A.1.AT.2000.Z01.E", {1: 7})

In [69]:
print(final_df)

   TIME_PERIOD                     PROVIDED_ID  \
0      2013-Q3  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
1      2013-Q4  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
2      2014-Q1  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
3      2014-Q2  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
4      2014-Q3  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
5      2014-Q4  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
6      2015-Q1  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
7      2015-Q2  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
8      2015-Q3  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
9      2015-Q4  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
10     2016-Q1  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
11     2016-Q2  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
12     2016-Q3  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
13     2016-Q4  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
14     2017-Q1  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
15     2017-Q2  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
16     2017-Q3  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
17     2017-Q4  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
18     2018-Q1  Q.HR.N.A.A20.A.1.AT.2000.Z01.E   
